# CSVs to Database considering normalization rules

In [3]:
import pandas as pd

## Games Details

In [4]:

game_details = pd.read_csv('../data/games_details.csv')
print(game_details.columns.values)

game_details.head()

['GAME_ID' 'TEAM_ID' 'TEAM_ABBREVIATION' 'TEAM_CITY' 'PLAYER_ID'
 'PLAYER_NAME' 'START_POSITION' 'COMMENT' 'MIN' 'FGM' 'FGA' 'FG_PCT'
 'FG3M' 'FG3A' 'FG3_PCT' 'FTM' 'FTA' 'FT_PCT' 'OREB' 'DREB' 'REB' 'AST'
 'STL' 'BLK' 'TO' 'PF' 'PTS' 'PLUS_MINUS']


,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,START_POSITION,COMMENT,MIN,FGM,...,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS
0,12000047,1610612766,CHA,Charlotte,1628998,Cody Martin,F,NaN,17:06,0.0,...,0.0,2.0,2.0,1.0,0.0,1.0,1.0,2.0,0.0,-31.0
1,12000047,1610612766,CHA,Charlotte,1629023,P.J. Washington,F,NaN,24:58,4.0,...,1.0,6.0,7.0,2.0,2.0,1.0,5.0,3.0,9.0,-2.0
2,12000047,1610612766,CHA,Charlotte,203469,Cody Zeller,C,NaN,22:45,5.0,...,2.0,2.0,4.0,0.0,0.0,1.0,0.0,2.0,13.0,-23.0
3,12000047,1610612766,CHA,Charlotte,1628984,Devonte' Graham,G,NaN,31:30,8.0,...,1.0,3.0,4.0,3.0,2.0,0.0,4.0,0.0,25.0,-7.0
4,12000047,1610612766,CHA,Charlotte,1626179,Terry Rozier,G,NaN,26:48,8.0,...,1.0,4.0,5.0,6.0,1.0,0.0,0.0,2.0,24.0,22.0


### Normalization

1) Primary Key 
  Composite  Key - Game ID, Player ID
2) Needs to be split into player Stats and Game Details
3) Team abbreviation seems duplicated

## Games

In [5]:
games = pd.read_csv('../data/games.csv')
print(games.columns.values)

games.head()

['GAME_DATE_EST' 'GAME_ID' 'GAME_STATUS_TEXT' 'HOME_TEAM_ID'
 'VISITOR_TEAM_ID' 'SEASON' 'TEAM_ID_home' 'PTS_home' 'FG_PCT_home'
 'FT_PCT_home' 'FG3_PCT_home' 'AST_home' 'REB_home' 'TEAM_ID_away'
 'PTS_away' 'FG_PCT_away' 'FT_PCT_away' 'FG3_PCT_away' 'AST_away'
 'REB_away' 'HOME_TEAM_WINS']


,GAME_DATE_EST,GAME_ID,GAME_STATUS_TEXT,HOME_TEAM_ID,VISITOR_TEAM_ID,SEASON,TEAM_ID_home,PTS_home,FG_PCT_home,FT_PCT_home,...,AST_home,REB_home,TEAM_ID_away,PTS_away,FG_PCT_away,FT_PCT_away,FG3_PCT_away,AST_away,REB_away,HOME_TEAM_WINS
0,2020-12-19,12000047,Final,1610612753,1610612766,2020,1610612753,120.0,0.433,0.792,...,23.0,50.0,1610612766,117.0,0.444,0.864,0.439,21.0,52.0,1
1,2020-12-19,12000048,Final,1610612764,1610612765,2020,1610612764,99.0,0.427,0.625,...,24.0,45.0,1610612765,96.0,0.402,0.647,0.326,18.0,51.0,1
2,2020-12-19,12000049,Final,1610612763,1610612737,2020,1610612763,116.0,0.400,0.744,...,21.0,43.0,1610612737,117.0,0.422,0.837,0.297,24.0,47.0,0
3,2020-12-18,12000039,Final,1610612754,1610612755,2020,1610612754,107.0,0.371,0.692,...,19.0,45.0,1610612755,113.0,0.533,0.629,0.355,23.0,48.0,0
4,2020-12-18,12000040,Final,1610612761,1610612748,2020,1610612761,105.0,0.380,0.737,...,27.0,37.0,1610612748,117.0,0.534,0.741,0.514,30.0,51.0,0


In [6]:
games['GAME_STATUS_TEXT'].unique()

array(['Final'], dtype=object)

In [7]:
games[games['SEASON']==2018].groupby(['GAME_DATE_EST']).size().reset_index(name='Count').sort_values(by='GAME_DATE_EST', ascending=False)

,GAME_DATE_EST,Count
227,2019-06-13,1
226,2019-06-10,1
225,2019-06-07,1
224,2019-06-05,1
223,2019-06-02,1
...,...,...
4,2018-10-02,5
3,2018-10-01,4
2,2018-09-30,4
1,2018-09-29,2


In [8]:
df = games[games['SEASON']==2018].groupby(['GAME_DATE_EST']).size().reset_index(name='Count').sort_values(by='GAME_DATE_EST', ascending=False)
df['GAME_DATE_EST'] = pd.to_datetime(df['GAME_DATE_EST'])
df['WEEK_NUM'] = df['GAME_DATE_EST'].dt.isocalendar().week
weekly_counts = df.groupby(['WEEK_NUM', df['GAME_DATE_EST'].dt.year]).agg({'Count':'sum'}).reset_index()
weekly_counts

,WEEK_NUM,GAME_DATE_EST,Count
0,1,2018,7
1,1,2019,42
2,2,2019,54
3,3,2019,46
4,4,2019,53
5,5,2019,46
6,6,2019,49
7,7,2019,28
8,8,2019,30
9,9,2019,55


### Normalization

1) Primary Key 
  Game ID
2) There should be a junction (to establish a many to many relationship) table between games and teams.

## Players

In [9]:
players = pd.read_csv('../data/players.csv')
print(players.columns.values)

players.head()

['PLAYER_NAME' 'TEAM_ID' 'PLAYER_ID' 'SEASON']


,PLAYER_NAME,TEAM_ID,PLAYER_ID,SEASON
0,Royce O'Neale,1610612762,1626220,2019
1,Bojan Bogdanovic,1610612762,202711,2019
2,Rudy Gobert,1610612762,203497,2019
3,Donovan Mitchell,1610612762,1628378,2019
4,Mike Conley,1610612762,201144,2019


### Normalization

1) Primary Key
   Player id
2) A table to have the historic of how a player moves from team to team across season

## Ranking

In [10]:
ranking = pd.read_csv('../data/ranking.csv')
print(ranking.columns.values)

ranking.head()

['TEAM_ID' 'LEAGUE_ID' 'SEASON_ID' 'STANDINGSDATE' 'CONFERENCE' 'TEAM' 'G'
 'W' 'L' 'W_PCT' 'HOME_RECORD' 'ROAD_RECORD' 'RETURNTOPLAY']


,TEAM_ID,LEAGUE_ID,SEASON_ID,STANDINGSDATE,CONFERENCE,TEAM,G,W,L,W_PCT,HOME_RECORD,ROAD_RECORD,RETURNTOPLAY
0,1610612747,0,12020,2020-12-21,West,L.A. Lakers,4,4,0,1.00,2-0,2-0,NaN
1,1610612762,0,12020,2020-12-21,West,Utah,3,3,0,1.00,2-0,1-0,NaN
2,1610612740,0,12020,2020-12-21,West,New Orleans,2,2,0,1.00,1-0,1-0,NaN
3,1610612745,0,12020,2020-12-21,West,Houston,4,3,1,0.75,2-0,1-1,NaN
4,1610612763,0,12020,2020-12-21,West,Memphis,4,3,1,0.75,1-1,2-0,NaN


### Normalization

1) Primary Key
   Composite Key - Team id, season id

In [11]:
ranking['SEASON_ID'].unique()

array([12020, 22019, 12019, 22013, 12013, 22012, 12012, 22011, 12011,
       22010, 12010, 22009, 12009, 22008, 12008, 22007, 12007, 22006,
       12006, 22005, 12005, 22004, 12004, 22003, 12003, 22002, 22018,
       12018, 22017, 12017, 22016, 12016, 22015, 12015, 22014, 12014])

## Teams

In [12]:
teams = pd.read_csv('../data/teams.csv')
print(teams.columns.values)
teams.head()

['LEAGUE_ID' 'TEAM_ID' 'MIN_YEAR' 'MAX_YEAR' 'ABBREVIATION' 'NICKNAME'
 'YEARFOUNDED' 'CITY' 'ARENA' 'ARENACAPACITY' 'OWNER' 'GENERALMANAGER'
 'HEADCOACH' 'DLEAGUEAFFILIATION']


,LEAGUE_ID,TEAM_ID,MIN_YEAR,MAX_YEAR,ABBREVIATION,NICKNAME,YEARFOUNDED,CITY,ARENA,ARENACAPACITY,OWNER,GENERALMANAGER,HEADCOACH,DLEAGUEAFFILIATION
0,0,1610612737,1949,2019,ATL,Hawks,1949,Atlanta,State Farm Arena,18729.0,Tony Ressler,Travis Schlenk,Lloyd Pierce,Erie Bayhawks
1,0,1610612738,1946,2019,BOS,Celtics,1946,Boston,TD Garden,18624.0,Wyc Grousbeck,Danny Ainge,Brad Stevens,Maine Red Claws
2,0,1610612740,2002,2019,NOP,Pelicans,2002,New Orleans,Smoothie King Center,NaN,Tom Benson,Trajan Langdon,Alvin Gentry,No Affiliate
3,0,1610612741,1966,2019,CHI,Bulls,1966,Chicago,United Center,21711.0,Jerry Reinsdorf,Gar Forman,Jim Boylen,Windy City Bulls
4,0,1610612742,1980,2019,DAL,Mavericks,1980,Dallas,American Airlines Center,19200.0,Mark Cuban,Donnie Nelson,Rick Carlisle,Texas Legends


### Normalization

1) Primary Key
   team id
2) Good Idea to split owener, general manager, headcoach to a "staff table"
3) Good idea to split arena's data to a table


In [13]:
teams['NICKNAME'].unique()

array(['Hawks', 'Celtics', 'Pelicans', 'Bulls', 'Mavericks', 'Nuggets',
       'Rockets', 'Clippers', 'Lakers', 'Heat', 'Bucks', 'Timberwolves',
       'Nets', 'Knicks', 'Magic', 'Pacers', '76ers', 'Suns',
       'Trail Blazers', 'Kings', 'Spurs', 'Thunder', 'Raptors', 'Jazz',
       'Grizzlies', 'Wizards', 'Pistons', 'Hornets', 'Cavaliers',
       'Warriors'], dtype=object)